# Things done these two weeks (2018-11-15)

## Generated training data using our own models

In [7]:
from piano_reduction import tools as pr, features as ft, compute_features as cf
from piano_reduction.score_data import ScoreData
import imp
imp.reload(pr)
imp.reload(ft)
imp.reload(cf)

<module 'piano_reduction.compute_features' from '/Users/jason2yik/fyp/piano-reduction-main/piano_reduction/compute_features.py'>

In [2]:
data = ScoreData.load('score_data/cosi_2.pkl')
testing_data, training_data = data.split()

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

x_train, y_train = training_data.generate_data(length=-1)
x_test, y_test = testing_data.generate_data(length=-1)
lr_0 = LogisticRegression().fit(x_train, y_train)
nb_0 = MultinomialNB().fit(x_train, y_train)

In [3]:
data = ScoreData.load('score_data/5.pkl')
data = data.get_y_pred(lr_0, length=-1)
data.df['y_train'] = data.df['y_pred']
data.df = data.df.query("(y_train == 1) or (x_train == 1)")
del data.df['y_pred']
data.save('own_output/lr/5.pkl')

In [14]:
data = ScoreData.load('own_output/lr/3.pkl')
data.show_score()

## Implemented new metrics

In [4]:
def jaccard_similarity(y_train, y_pred):
    s = set()
    t = set()
    for i in range(0, len(y_train)):
        for j in range(0, 128):
            if y_train[i, j] == 1:
                t.add((i, j))
    for i in range(0, len(y_pred)):
        for j in range(0, 128):
            if y_pred[i, j] == 1:
                s.add((i, j))
    return len(s&t) * 1.0 / (len(s) + len(t) - len(s&t))

def pitch_class_jaccard_similarity(y_train, y_pred):
    s = set()
    t = set()
    for i in range(0, len(y_train)):
        for j in range(0, 128):
            if y_train[i, j] == 1:
                t.add((i, j % 12))
    for i in range(0, len(y_pred)):
        for j in range(0, 128):
            if y_pred[i, j] == 1:
                s.add((i, j % 12))
    return len(s&t) * 1.0 / (len(s) + len(t) - len(s&t))

In [5]:
data = ScoreData.load('own_output/lr/3.pkl')
testing_data, training_data = data.split()
x_train, y_train = training_data.generate_data(length=-1)
x_test, y_test = testing_data.generate_data(length=-1)
lr = LogisticRegression().fit(x_train, y_train)
nb = MultinomialNB().fit(x_train, y_train)
nn = pr.dense()
nn.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[pr.early_stop()], verbose=0)

x_train, y_train = training_data.generate_data(length=1)
x_test, y_test = testing_data.generate_data(length=1)
lstm_1 = pr.lstm()
lstm_1.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[pr.early_stop()], verbose=0)

x_train, y_train = training_data.generate_data(length=5)
x_test, y_test = testing_data.generate_data(length=5)
lstm_5 = pr.lstm()
lstm_5.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[pr.early_stop()], verbose=0)

x_train, y_train = training_data.generate_data(length=10)
x_test, y_test = testing_data.generate_data(length=10)
lstm_10 = pr.lstm()
lstm_10.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[pr.early_stop()], verbose=0)

class KeepAll():
    @classmethod
    def predict(cls, x):
        return [1 for _ in range(x.shape[0])]
class KeepSome():
    @classmethod
    def predict(cls, x):
        from random import randint
        return [randint(0, 1) for _ in range(x.shape[0])]

In [10]:
x_train, y_train = training_data.generate_data(length=1)
x_test, y_test = testing_data.generate_data(length=1)
gru_1 = pr.gru()
gru_1.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[pr.early_stop()], verbose=0)

x_train, y_train = training_data.generate_data(length=5)
x_test, y_test = testing_data.generate_data(length=5)
gru_5 = pr.gru()
gru_5.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[pr.early_stop()], verbose=0)

x_train, y_train = training_data.generate_data(length=10)
x_test, y_test = testing_data.generate_data(length=10)
gru_10 = pr.gru()
gru_10.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[pr.early_stop()], verbose=0)

In [75]:
print('%30s %20s %20s %20s %20s' % ('', 'Jaccard Similarity', 'Ignore Octave', 'Accuracy', 'F1 score'))
models = [('Keep All', KeepAll, -1),
          ('Keep Some', KeepSome, -1),
          ('Logistic Regression', lr, -1), 
          ('Naive Bayes', nb, -1),
          ('NN', nn, -1),
          ('LSTM with 1', lstm_1, 1),
          ('LSTM with 5', lstm_5, 5),
          ('LSTM with 10', lstm_10, 10),
          ('GRU with 1', gru_1, 1),
          ('GRU with 5', gru_5, 5),
          ('GRU with 10', gru_10, 10),
         ]
from sklearn import metrics
for name, model, length in models:
    data = ScoreData.load('own_output/lr/2.pkl')
    validation_data, _ = data.split(0.5)
    validation_data = data
    validation_data = validation_data.get_y_pred(model, length, threshold=0.5)
    tmptrain = validation_data.to_binary('y_train')
    tmppred = validation_data.to_binary('y_pred')
    print('%30s %20.8f %20.8f %20.8f %20.8f' % (name,
                                jaccard_similarity(tmptrain, tmppred),
                                pitch_class_jaccard_similarity(tmptrain, tmppred),
                                metrics.accuracy_score(validation_data.df['y_train'], validation_data.df['y_pred']), 
                                metrics.f1_score(validation_data.df['y_train'], validation_data.df['y_pred'])
                               ))
    

                                 Jaccard Similarity        Ignore Octave             Accuracy             F1 score
                      Keep All           0.81160365           0.92913386           0.81160365           0.89600576
                     Keep Some           0.46137184           0.65997323           0.51368970           0.63142292
           Logistic Regression           0.97187500           0.98194444           0.97653194           0.98573693
                   Naive Bayes           0.84392265           0.92980132           0.85267275           0.91535581
                            NN           0.98416469           0.99020979           0.98696219           0.99201915
                   LSTM with 1           0.97100313           0.98467967           0.97588005           0.98528827
                   LSTM with 5           0.97470356           0.98603352           0.97913950           0.98718975
                  LSTM with 10           0.95746326           0.97520661        

In [12]:
data = ScoreData.load('own_output/lr/3.pkl')
data = data.get_y_pred(lstm_10, 10)
data.save('intermediate/3.pkl')

## Implemented a basic postprocessor

In [76]:
data = ScoreData.load('intermediate/3.pkl')


In [57]:
data_pred = data.to_binary('y_pred')
for i in range(0,len(data_pred)):
    cnt = 0
    highest = -1
    lowest = 129
    for j in range(0,128):
        if data_pred[i][j] == 1:
            highest = max(highest,j)
            lowest = min(lowest,j)
    for j in range(60,128):
        if data_pred[i][j] ==1 and j < highest-12:
            data_pred[i][j] = 0
            data_pred[i][j+12] = 1
    for j in reversed(range(0, 60)):
        if data_pred[i][j] == 1 and j > lowest + 12:
            data_pred[i][j] = 0
            data_pred[i][j - 12] = 1

In [77]:
data_pred = data.to_binary('y_pred')
for i in range(0,len(data_pred)):
    cnt = 0
    highest = -1
    lowest = -1
    for j in range(0,128):
        if data_pred[i][j] == 1:
            highest = max(highest,j)
            if j < 60:
                lowest = max(lowest,j)
    for j in range(60,128):
        if data_pred[i][j] ==1 and j < highest-12:
            data_pred[i][j] = 0
            data_pred[i][j+12] = 1
    for j in range(0, 60):
        if data_pred[i][j] == 1 and j < lowest - 12:
            data_pred[i][j] = 0
            data_pred[i][j + 12] = 1

In [78]:
new_data = data.merge_binary(data_pred, name='post', skip_features=True)
new_data.show_score(col=['x_train', 'y_train', 'y_pred', 'post'])

## Continued experimenting binary vectors

In [51]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
model2 = Sequential()
model2.add(LSTM(128, input_shape=(None, 128)))
model2.add(Dropout(0.2))
model2.add(Activation('sigmoid'))
model2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', pr.f1])
model2.summary()

model3 = Sequential()
model3.add(GRU(128, input_shape=(None, 128)))
model3.add(Dropout(0.2))
model3.add(Activation('sigmoid'))
model3.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', pr.f1])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_21 (Dropout)         (None, 128)               0         
_________________________________________________________________
activation_23 (Activation)   (None, 128)               0         
Total params: 131,584
Trainable params: 131,584
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_14 (GRU)                 (None, 128)               98688     
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0         
_________________________________________________________________
acti

In [52]:
from random import randint
data = ScoreData.load('own_output/lr/3.pkl')

for _ in range(10):
    length = randint(1, 6)
    if _ == 0:
        length = 1
    key = randint(-12, 12)
    testing_data, training_data = data.split()
    training_data.df['ps'] -= key
    x_train, y_train = training_data.generate_data_2(length=length)
    x_test, y_test = testing_data.generate_data_2(length=length)
    print('length = %d, key = %d' % (length, key))
    logger = pr.NBatchLogger(display=200)
    model2.fit(x_train, y_train, epochs=200, batch_size=64, validation_data=(x_test, y_test), callbacks=[logger], verbose=0)
    logger = pr.NBatchLogger(display=200)
    model3.fit(x_train, y_train, epochs=200, batch_size=64, validation_data=(x_test, y_test), callbacks=[logger], verbose=0)

length = 1, key = 2
step: 200/200 ...  - loss: 0.4475 - acc: 0.9830 - f1: 0.3690 - val_loss: 0.4417 - val_acc: 0.9802 - val_f1: 0.2277
step: 200/200 ...  - loss: 0.3910 - acc: 0.9834 - f1: 0.4203 - val_loss: 0.3777 - val_acc: 0.9800 - val_f1: 0.2487
length = 2, key = -5
step: 200/200 ...  - loss: 0.3609 - acc: 0.9864 - f1: 0.4828 - val_loss: 0.3397 - val_acc: 0.9834 - val_f1: 0.3366
step: 200/200 ...  - loss: 0.3481 - acc: 0.9901 - f1: 0.6703 - val_loss: 0.3264 - val_acc: 0.9871 - val_f1: 0.5694
length = 6, key = 0
step: 200/200 ...  - loss: 0.3521 - acc: 0.9861 - f1: 0.4647 - val_loss: 0.3282 - val_acc: 0.9843 - val_f1: 0.3990
step: 200/200 ...  - loss: 0.3450 - acc: 0.9924 - f1: 0.7695 - val_loss: 0.3226 - val_acc: 0.9906 - val_f1: 0.7351
length = 4, key = 4
step: 200/200 ...  - loss: 0.3521 - acc: 0.9858 - f1: 0.4459 - val_loss: 0.3274 - val_acc: 0.9849 - val_f1: 0.4289
step: 200/200 ...  - loss: 0.3451 - acc: 0.9923 - f1: 0.7656 - val_loss: 0.3222 - val_acc: 0.9909 - val_f1: 0.7573

### GRU train faster than LSTM

In [79]:
data = ScoreData.load('score_data/4.pkl')
from sklearn import metrics
print('%30s %20s %15s %15s %15s %15s' % ('', 'Jaccard Similarity', 'Ignore Octave', 'Accuracy', 'F1 score', 'roc_auc_score'))
for name, model in [('LSTM', model2), ('GRU', model3)]:
    for length in range(1, 9):
        validation_data = data
        validation_data = validation_data.merge_binary((model.predict(validation_data.generate_data_2(length=length)[0]) > 0.5) * 1, skip_features=True)
        tmptrain = validation_data.to_binary('y_train')
        tmppred = validation_data.to_binary('y_pred')
        print('%30s %20.8f %15.8f %15.8f %15.8f %15.8f' % ('%s with %d' % (name, length),
                                    jaccard_similarity(tmptrain, tmppred),
                                    pitch_class_jaccard_similarity(tmptrain, tmppred),
                                    metrics.accuracy_score(validation_data.df['y_train'], validation_data.df['y_pred']), 
                                    metrics.f1_score(validation_data.df['y_train'], validation_data.df['y_pred']),
                                    metrics.roc_auc_score(validation_data.to_binary('y_train').flatten(), model.predict(validation_data.generate_data_2(length=length)[0]).flatten())
                                   ))
    print()

                                 Jaccard Similarity   Ignore Octave        Accuracy        F1 score   roc_auc_score
                   LSTM with 1           0.31130614      0.48346264      0.41848300      0.47480315      0.81262274
                   LSTM with 2           0.32707845      0.50595972      0.43722755      0.49293009      0.86882672
                   LSTM with 3           0.31438302      0.48847737      0.43034393      0.47837353      0.87853254
                   LSTM with 4           0.31621692      0.49052718      0.43236190      0.48049363      0.86222058
                   LSTM with 5           0.31294490      0.48405797      0.43360139      0.47670683      0.86368147
                   LSTM with 6           0.31157505      0.48277293      0.43344933      0.47511586      0.85686300
                   LSTM with 7           0.30897538      0.47797174      0.43211488      0.47208738      0.84010818
                   LSTM with 8           0.30693856      0.47609148     

In [69]:
data = ScoreData.load('own_output/lr/2.pkl')
validation_data = data


0.9413706502967256

In [55]:
model2.save('models/20181115/lstm.h5')
model3.save('models/20181115/gru.h5')

In [61]:
data = ScoreData.load('own_output/lr/2.pkl')
data = data.merge_binary((model3.predict(validation_data.generate_data_2(length=6)[0]) > 0.5) * 1, skip_features=True)
data.show_score()